<h3>  &nbsp;&nbsp;Train on GPU&nbsp;&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a></h3>

1. On the main menu, click Runtime and select **Change runtime type**. Set "GPU" as the hardware accelerator.
1. Click Runtime again and select **Runtime > Run All**. You can also run the cells manually with Shift-ENTER. 

In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [2]:
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers 
import tensorflow.keras.optimizers as optimizers

from keras.datasets import cifar10
from keras.utils import to_categorical

import numpy as np

In [3]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# MODIFY THE CODE TO ADD A VALIDATION SET

validation_images = train_images[40000:]
validation_labels = train_labels[40000:]

train_images = train_images[:40000]
train_labels = train_labels[:40000]

In [4]:
# CHECKER:

print("Training set: images {:}\tlabels: {:}".format(train_images.shape, train_labels.shape ) )
print("validation set: images {:}\tlabels: {:}".format(validation_images.shape, validation_labels.shape ) )
print("Test set: images {:}\tlabels: {:}".format(test_images.shape, test_labels.shape ) )

Training set: images (40000, 32, 32, 3)	labels: (40000, 1)
validation set: images (10000, 32, 32, 3)	labels: (10000, 1)
Test set: images (10000, 32, 32, 3)	labels: (10000, 1)


In [5]:
train_images = train_images.reshape((40000, 32, 32, 3))
train_images = train_images.astype('float32') / 255

validation_images = validation_images.reshape((10000, 32, 32, 3))
validation_images = validation_images.astype('float32') / 255

test_images = test_images.reshape((10000, 32, 32, 3))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
validation_labels = to_categorical(validation_labels)
test_labels = to_categorical(test_labels)

In [6]:
print("Training set: images {:}\tlabels: {:}".format(train_images.shape, train_labels.shape ) )
print("validation set: images {:}\tlabels: {:}".format(validation_images.shape, validation_labels.shape ) )
print("Test set: images {:}\tlabels: {:}".format(test_images.shape, test_labels.shape ) )

Training set: images (40000, 32, 32, 3)	labels: (40000, 10)
validation set: images (10000, 32, 32, 3)	labels: (10000, 10)
Test set: images (10000, 32, 32, 3)	labels: (10000, 10)


In [12]:
from tensorflow.keras.applications.resnet import ResNet152 

conv_base = ResNet152(include_top=False, weights='imagenet',
                             input_shape=(32, 32, 3), 
                             pooling='avg')


conv_base.summary()

234700800/234698864 [==============================] - 82s 0us/step
Model: "resnet152"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
______________________

Total params: 58,370,944
Trainable params: 58,219,520
Non-trainable params: 151,424
__________________________________________________________________________________________________


## Features extraction

In [13]:
def extract_features(input_data):
    return conv_base.predict(input_data) 

train_features = extract_features(train_images)
validation_features = extract_features(validation_images)
test_features = extract_features(test_images)

In [14]:
print("train features:", train_features.shape)
print("validation features:", validation_features.shape)
print("test features:", test_features.shape)

train features: (40000, 2048)
validation features: (10000, 2048)
test features: (10000, 2048)


In [ ]:
STOOP... think about the size!

In [ ]:
train_features_vectorized = np.reshape(train_features, (len(train_labels) , 512))
validation_features_vectorized = np.reshape(validation_features, (len(validation_labels),512))
test_features_vectorized = np.reshape(test_features, (len(test_labels), 512))

print("train features:", train_features_vectorized.shape)
print("validation features:", validation_features_vectorized.shape)
print("test features:", test_features_vectorized.shape)

In [22]:
model = models.Sequential()

#model.add(layers.Conv2D(32, (3,3), activation='relu', padding = 'same', input_shape=(32, 32, 3)))
#model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Dense(128, activation='relu', input_dim= 2048))
model.add(layers.Dropout(0.5))

#model.add(layers.Dense(128, activation='relu'))
#model.add(layers.Dropout(0.5))

#model.add(layers.Dense(128, activation='relu'))
#model.add(layers.Dropout(0.5))

model.add(layers.Dense(10, activation='softmax'))


model.compile(optimizer=optimizers.RMSprop(lr=1e-2),
              loss='categorical_crossentropy',
                  metrics=['accuracy'])


model_hist = model.fit(train_features, train_labels,
                    epochs=100,
                    batch_size=1024,
                    validation_data=(validation_features, validation_labels))

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 3s 81us/sample - loss: 2.5780 - accuracy: 0.1031 - val_loss: 2.3007 - val_accuracy: 0.1287
Epoch 2/100
40000/40000 [==============================] - 2s 62us/sample - loss: 2.3053 - accuracy: 0.1010 - val_loss: 2.3026 - val_accuracy: 0.1024
Epoch 3/100
40000/40000 [==============================] - 3s 70us/sample - loss: 2.3063 - accuracy: 0.0990 - val_loss: 2.3007 - val_accuracy: 0.1032
Epoch 4/100
40000/40000 [==============================] - 3s 63us/sample - loss: 2.3023 - accuracy: 0.1076 - val_loss: 2.2869 - val_accuracy: 0.1333
Epoch 5/100
40000/40000 [==============================] - 3s 64us/sample - loss: 2.2939 - accuracy: 0.1190 - val_loss: 2.3019 - val_accuracy: 0.1013
Epoch 6/100
40000/40000 [==============================] - 2s 58us/sample - loss: 2.2931 - accuracy: 0.1184 - val_loss: 2.3096 - val_accuracy: 0.1077
Epoch 7/100
40000/40000 [=========================

Epoch 55/100
40000/40000 [==============================] - 3s 63us/sample - loss: 2.2152 - accuracy: 0.1662 - val_loss: 2.1564 - val_accuracy: 0.1965
Epoch 56/100
40000/40000 [==============================] - 2s 58us/sample - loss: 2.2120 - accuracy: 0.1693 - val_loss: 2.1837 - val_accuracy: 0.2010
Epoch 57/100
40000/40000 [==============================] - 3s 64us/sample - loss: 2.2118 - accuracy: 0.1707 - val_loss: 2.1619 - val_accuracy: 0.1807
Epoch 58/100
40000/40000 [==============================] - 2s 61us/sample - loss: 2.2136 - accuracy: 0.1695 - val_loss: 2.2805 - val_accuracy: 0.1447
Epoch 59/100
40000/40000 [==============================] - 2s 58us/sample - loss: 2.2111 - accuracy: 0.1706 - val_loss: 2.1784 - val_accuracy: 0.1784
Epoch 60/100
40000/40000 [==============================] - 3s 63us/sample - loss: 2.2134 - accuracy: 0.1673 - val_loss: 2.1788 - val_accuracy: 0.1763
Epoch 61/100
40000/40000 [==============================] - 2s 61us/sample - loss: 2.2115 - ac

In [ ]:
model_hist = model.fit(train_features, train_labels,
                    epochs=100,
                    batch_size=1024,
                    validation_data=(validation_features, validation_labels))

In [ ]:
model_hist = model.fit(train_features, train_labels,
                    epochs=100,
                    batch_size=1024,
                    validation_data=(validation_features, validation_labels))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

train_loss = model_hist.history['loss']
val_loss= model_hist.history['val_loss']

epochs_val = range(0, len(train_loss))
epochs = range(0, len(train_loss))

plt.plot(epochs_val, val_loss, 'b+', label='Validation', c = 'purple')
plt.plot(epochs, train_loss, 'bo', label='training', c ='green')
plt.xlabel('Epochs')
plt.ylabel('Validation loss')
plt.legend()

plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_features, test_labels)
print('test_acc:', test_acc)
test_loss

In [ ]:
model.save_weights('./cifar10_VGG16.h5', overwrite=True)